In [235]:
#import voor alle benodigde libraries
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns
import plotly.express as px
from sodapy import Socrata
import folium

Importing Gekentekende Voertuigen

The source of our API for the dataframe 'voertuigen_df' has their own visualizations.
From these visualizations we can determine that the following columns do not have more than 50% NaN values:

In [6]:
# #Inladen API voertuigen
# #Van de site:
# #https://opendata.rdw.nl/Voertuigen/Open-Data-RDW-Gekentekende_voertuigen/m9d7-ebf2
# client_1 = Socrata("opendata.rdw.nl", None)

# voertuigen = client_1.get("m9d7-ebf2", limit=15200000, where='laadvermogen>0',
#                           select='kenteken, voertuigsoort, merk, handelsbenaming, inrichting, eerste_kleur, aantal_cilinders, massa_ledig_voertuig, wam_verzekerd, aantal_deuren, aantal_wielen, europese_voertuigcategorie, vermogen_massarijklaar, datum_tenaamstelling_dt, datum_eerste_toelating_dt, datum_eerste_tenaamstelling_in_nederland_dt'
#                           )

# voertuigen_df = pd.DataFrame.from_records(voertuigen)

# voertuigen_df.shape

Importing Brandstof Types Voertuigen

The source of our API for the dataframe 'brandstof_df' has their own visualizations.
From these visualizations we can determine that the following columns do not have more than 50% NaN values:

In [7]:
# #Inladen API brandstof types
# #Van de site:
# #https://opendata.rdw.nl/Voertuigen/Open-Data-RDW-Gekentekende_voertuigen_brandstof/8ys7-d773
# client_2 = Socrata("opendata.rdw.nl", None)

# brandstof = client_2.get("8ys7-d773", limit=1300000, where="brandstof_omschrijving='Elektriciteit'",
#                          select='kenteken, brandstof_volgnummer, brandstof_omschrijving, emissiecode_omschrijving, milieuklasse_eg_goedkeuring_licht, klasse_hybride_elektrisch_voertuig, uitlaatemissieniveau'
#                          )

# brandstof_df = pd.DataFrame.from_records(brandstof)

# brandstof_df.shape

Merging voertuigen_df & brandstof_df so we have a dataframe that consists solely of electric vehicles with all their data.

In [8]:
# elektrisch_df = voertuigen_df.merge(brandstof_df, how='inner', on='kenteken')

In [9]:
# elektrisch_df.shape

In [10]:
# elektrisch_df.to_csv(r'C:\School\Y3\Semester 2 - Minor\Block 1\Case3\elektrischV2.csv')

Importing an API that provides data on charging docks for cars in the Netherlands

In [380]:
#Inladen API chargemap
response = requests.get("https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=1000&compact=true&verbose=false&key=6686970f-0aa6-4cb5-ae81-cf8f6389a816")

#Omzetten naar dictionary
responsejson  = response.json()

#Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
charge_map = pd.json_normalize(responsejson)

charge_map_df = pd.DataFrame.from_records(charge_map)

#Dropping unnecessary columns:
charge_map_df = charge_map_df.drop(columns=['IsRecentlyVerified', 'UUID', 'DataProviderID', 'AddressInfo.CountryID', 
                            'AddressInfo.DistanceUnit', 'AddressInfo.RelatedURL', 'AddressInfo.ContactTelephone1', 
                            'AddressInfo.ContactEmail', 'AddressInfo.ContactTelephone2', 'OperatorsReference', 
                            'DataProvidersReference', 'GeneralComments', 'AddressInfo.AddressLine2',
                            'AddressInfo.AccessComments'
                            ])

#Renaming column names:
charge_map_df.rename(columns={'AddressInfo.Latitude':'LAT', 'AddressInfo.Longitude':'LON', 
                              'AddressInfo.Postcode':'Postcode', 'AddressInfo.ID':'Address_ID', 
                              'AddressInfo.Title':'Address_Title', 'AddressInfo.AddressLine1':'Address_Line1',
                              'AddressInfo.StateOrProvince':'Address_StateOrProvince', 
                              'AddressInfo.Town':'Address_Town'},
                              inplace=True)                            

In [182]:
laadpaal_df = pd.read_csv('laadpaaldata.csv')
elektrisch_df = pd.read_csv('elektrischV2.csv', index_col=0)

In [58]:
#Restaurants = MCDonald's / Mc Donald's,
#Recreatie = hotel, Gelredome
#Parkeren = P+R, parkeerterrein
#Tankstations = Fastned, Supercharger (Tesla),

In [348]:
#Creating address types to make color coding possible.

#Recreation type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Hotel', case=False), 'Address_Type'] = 'Recreatie'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Gelredome', case=False), 'Address_Type'] = 'Recreatie'

#Tankstation type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Fastned', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Supercharger', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Shell', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('IONITY', case=False), 'Address_Type'] = 'Tankstation'

#Restaurants type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Mc Donald', case=False), 'Address_Type'] = 'Restaurant'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('McDonald', case=False), 'Address_Type'] = 'Restaurant'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Restaurant', case=False), 'Address_Type'] = 'Restaurant'

#Parkeerplaats type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Parkeer', case=False), 'Address_Type'] = 'Parkeerterrein'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Garage', case=False), 'Address_Type'] = 'Parkeerterrein'

#Filling in the NaNs with 'Straat'
charge_map_df['Address_Type'].fillna('Straat', inplace=True)

In [333]:
def color_type(type):
    if type == 'Recreatie':
        return 'green'
    elif type == 'Straat':
        return 'blue'
    elif type == 'Tankstation':
        return 'black'
    elif type == 'Restaurant':
        return 'orange'
    elif type =='Parkeerterrein':
        return 'darkblue'

#Test code
color_type(charge_map_df['Address_Type'][3])

'orange'

In [381]:
# r'\D+' = only numerical (1-9)
# r'[^0-9]+' = only numerical (1-9)

charge_map_df['Postcode'] = charge_map_df['Postcode'].fillna(0)
charge_map_df['Postcode'] = charge_map_df['Postcode'].str.replace(r'[^0-9]+', '')

charge_map_df['Postcode'] = pd.to_numeric(charge_map_df['Postcode'])
charge_map_df['Postcode'] = charge_map_df['Postcode'].fillna(0)
# charge_map_df['Postcode'] = charge_map_df['Postcode'].astype(int)
#charge_map_df['Postcode'] = int(charge_map_df['Postcode'])

charge_map_df['Postcode'].astype(str).astype(int)
#charge_map_df['Postcode'] = charge_map_df['Postcode'].round()

C:\Users\nikay\AppData\Local\Temp\ipykernel_15292\2071251785.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  charge_map_df['Postcode'] = charge_map_df['Postcode'].str.replace(r'[^0-9]+', '')


ValueError: invalid literal for int() with base 10: '8251.0'

In [363]:
charge_map_df['Postcode'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000 entries, 0 to 999
Series name: Postcode
Non-Null Count  Dtype  
--------------  -----  
963 non-null    float64
dtypes: float64(1)
memory usage: 7.9 KB


In [369]:
charge_map_df.head(10)

,DateLastVerified,ID,OperatorID,UsageTypeID,Connections,NumberOfPoints,StatusTypeID,DateLastStatusUpdate,DataQualityLevel,DateCreated,SubmissionStatusTypeID,Address_ID,Address_Title,Address_Line1,Address_StateOrProvince,Postcode,LAT,LON,Address_Town,UsageCost
0,2023-03-15T11:19:00Z,256957,1.0,6.0,[],1.0,50,2023-03-15T11:19:00Z,1,2023-03-15T10:20:00Z,200,257341,De Wildkansel,De Wildkansel,Flevoland,8251.0,52.521086,5.731164,NaN,NaN
1,2023-03-04T15:56:00Z,238390,47.0,4.0,"[{'ID': 392175, 'ConnectionTypeID': 33, 'Statu...",1.0,50,2023-03-04T15:56:00Z,1,2023-03-04T15:56:00Z,200,238774,Mc Donalds Ijsselstein,Lorentzlaan,Utrecht,3401.0,52.014896,5.062458,IJsselstein,NaN
2,2023-03-04T15:52:00Z,238389,47.0,4.0,"[{'ID': 392173, 'ConnectionTypeID': 33, 'Statu...",6.0,50,2023-03-04T15:52:00Z,1,2023-03-04T15:52:00Z,200,238773,Haje Allego Ultra E Nieuwegein Oost,Waterliniedok,Utrecht,3433.0,52.011182,5.122047,Nieuwegein,NaN
3,2023-03-04T15:49:00Z,238388,47.0,4.0,"[{'ID': 392171, 'ConnectionTypeID': 33, 'Statu...",1.0,50,2023-03-04T15:49:00Z,1,2023-03-04T15:49:00Z,200,238772,Mc Donalds Nieuwegein,Ravenswade,Utrecht,3439.0,52.049201,5.117389,Nieuwegein,NaN
4,2023-03-04T14:27:00Z,238387,47.0,4.0,"[{'ID': 392170, 'ConnectionTypeID': 33, 'Statu...",2.0,50,2023-03-04T14:27:00Z,1,2023-03-04T14:27:00Z,200,238771,Divers Woerden,Europabaan,Utrecht,3446.0,52.075588,4.901184,Woerden,NaN
5,2023-03-04T14:16:00Z,238386,47.0,4.0,"[{'ID': 392167, 'ConnectionTypeID': 33, 'Statu...",2.0,50,2023-03-04T14:16:00Z,1,2023-03-04T14:16:00Z,200,238770,Shell A12 Hellevliet,Rijksweg A12,Utrecht,3481.0,52.080940,4.983936,Woerden,NaN
6,2023-03-04T14:11:00Z,238385,47.0,4.0,"[{'ID': 392165, 'ConnectionTypeID': 33, 'Statu...",2.0,50,2023-03-04T14:11:00Z,1,2023-03-04T14:07:00Z,200,238769,Shell Recharge Veldhuizen,Veldhuizerweg,Utrecht,3481.0,52.081911,5.003279,Harmelen,NaN
7,2023-02-23T20:59:00Z,235545,3387.0,4.0,"[{'ID': 388570, 'ConnectionTypeID': 25, 'Refer...",2.0,50,2023-02-23T20:59:00Z,1,2023-02-23T20:42:00Z,200,235929,Rivierenlaan 164,Rivierenlaan 164,Zuid-Holland,2641.0,52.008438,4.438623,Pijnacker,NaN
8,2023-02-21T13:18:00Z,235341,103.0,1.0,"[{'ID': 388152, 'ConnectionTypeID': 25, 'Refer...",2.0,50,2023-02-21T13:18:00Z,1,2023-02-18T09:43:00Z,200,235725,Rosendaalsestraat 318,Rosendaalsestraat 318,NaN,6824.0,51.989284,5.926503,Arnhem,NaN
9,2023-02-11T08:31:00Z,213619,45.0,1.0,"[{'ID': 355834, 'ConnectionTypeID': 33, 'Refer...",2.0,50,2023-02-11T08:31:00Z,1,2023-02-08T14:38:00Z,200,213999,Avia Marees Den Oever,Havenweg 2,Noord Holland,1779.0,52.929068,5.028645,Den Oever,€0.65 per kWh


In [174]:
#  ik heb een functie gevonden op het internet voor het toevoegen van een categorische legenda:
# (bron: https://stackoverflow.com/questions/65042654/how-to-add-categorical-legend-to-python-folium-map)

def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

In [220]:
#Generating the map
map = folium.Map(location=[52.0893191, 5.1101691], zoom_start=8)

#Adding groups
post10 = folium.FeatureGroup(name="postcodes: 1000 - 1999").add_to(map)
post20 = folium.FeatureGroup(name="postcodes: 2000 - 2999", show=False).add_to(map)
post30 = folium.FeatureGroup(name="postcodes: 3000 - 3999", show=False).add_to(map)
post40 = folium.FeatureGroup(name="postcodes: 4000 - 4999", show=False).add_to(map)
post50 = folium.FeatureGroup(name="postcodes: 5000 - 5999", show=False).add_to(map)
post60 = folium.FeatureGroup(name="postcodes: 6000 - 6999", show=False).add_to(map)
post70 = folium.FeatureGroup(name="postcodes: 7000 - 7999", show=False).add_to(map)
post80 = folium.FeatureGroup(name="postcodes: 8000 - 8999", show=False).add_to(map)
post90 = folium.FeatureGroup(name="postcodes: 9000 - 9999", show=False).add_to(map)

#Adding markers
for index, row in charge_map_df.iterrows():
    # if row['Postcode'] < 2000:
      folium.Marker(location=[row['LAT'], row['LON']], popup=row['Address_Title'], icon=folium.Icon(color=color_type(row['Address_Type']))).add_to(map)

#Adding legend
map = add_categorical_legend(map, 'Type laadpunten',
                             colors = ['blue', 'green', 'black', 'orange', 'darkblue'],
                             labels = ['Straat', 'Recreatie', 'Tankstation', 'Restaurant', 'Parkeerterrein']
                             )

# Layer control toevoegen zodat de postcode te selecteren is
folium.LayerControl(position='bottomleft', collapsed=False).add_to(map)

map